In [2]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import date, datetime, timedelta
from datetime import datetime

def soltrsupcol(M,n):
    z = n.copy()
    k = np.max(np.nonzero(n))
    for j in reversed(range(k + 1)):
        z[j] = z[j] / M[j,j]
        z[:j] = z[:j] - z[j] * M[: j, j]
    return z

def hh_vector(x):
    
    #u, rho = hh_vector(x)
    #Calcula u y rho tal que Q = I - rho u u^T
    #cumple Qx = \|x\|_2 e^1
    
    n = len(x)
    rho = 0
    u = x.copy()
    u[0] = 1.

    if n == 1:
        sigma = 0
    else:
        sigma = np.sum(x[1:]**2)

    if sigma>0 or x[0]<0:
        mu = np.sqrt(x[0]**2 + sigma)
        if x[0]<=0:
            gamma = x[0] - mu
        else:
            gamma = -sigma/(x[0] + mu)

        rho = 2*gamma**2/(gamma**2 + sigma)
        u = u/gamma
        u[0] = 1

    return u, rho

def qrhholder(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    p = min(m, n)

    for j in range(p):
        # I = j:, J = j:
        u, rho = hh_vector(R[j:, j])
        w = rho * u
        R[j:, j:] = R[j:, j:] - np.outer(w, u.T @ R[j:, j:])
        Q[:, j:] = Q[:, j:] - Q[:, j:] @ np.outer(w, u)

    return Q, R
    
def sol_cuadmin(A, b):
    m, n = A.shape

    #Asumimos que A tiene rango completo
    p = np.min([m,n])
    y_sol = np.zeros(n)
    # I = {1, ... , p} -> :p
    # J = {1, ... , p} \ I = {p+1, ... , n} -> p:

    Q, R = qrhholder(A)

    q = Q.T @ b

    y_sol[:p] = soltrsupcol(R[ :p, :p], q[:p])

    residuo = np.linalg.norm(q[p:])

    return y_sol, residuo

In [41]:
#Funcion que me da la Matriz de Datos de los precios de cierre en la ventana temporal indicada

# Formato fecha debe estar en mm-dd-yy

def Matriz_Precios(Tickers, start_date, end_date):

    #Convierto la fecha de clase str a clase date
    end_date =  datetime.strptime(end_date, '%Y-%m-%d')
    start_date = datetime.strptime(start_date, '%Y-%m-%d')

    #Busi = pd.date_range(start_date.date(), end_date.date(), freq='B')

    #Obtengo la cantidad de dias que tengo el dato de precio de las acciones
    N_columnas = len(yf.download('MSFT', start_date , end_date)['Close'])
    #Armo la matriz de precios, los cuales ire agregando
    A = np.zeros((len(Tickers), N_columnas))

    i = 0
    for company in Tickers:
        ith_company = yf.download(company, start_date , end_date)['Close']
        A[ i , :] = ith_company
        i += 1
    return(A)

Tickers = ['AAPL', 'MELI', 'MSFT', 'AMZN', 'KO', 'GOOGL', 'GS', 'DE', 'JPM', 'IBM', 'TSLA', 'AMD', 'GGAL', 'BBAR']

Precios = Matriz_Precios(Tickers, '2015-01-01', '2016-01-01')
print(Precios.shape)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [43]:
#DMD
X1 = Precios[ : , : 17]
X2 = Precios[ : ,1 : 18]

def dmd(X1, X2, truncate=None):

    U2,Sig2,Vh2 = np.linalg.svd(X1, False) # SVD of input matrix

    r = len(Sig2) if truncate is None else truncate # rank truncation

    U = U2[:,:r]                    #
    Sig = np.diag(Sig2)[:r,:r]      #Trunco las matrices de la descomposicion SVD
    V = Vh2.conj().T[:,:r]          #

    Atil = U.conj().T @ X2 @ V @ np.linalg.inv(Sig) # build A tilde

    mu ,W = np.linalg.eig(Atil)

    Phi = X2 @ V @ np.linalg.inv(Sig) @ W # build DMD modes

    return mu, Phi

mu, Phi = dmd(X1,X2)

def x(t, Phi, mu, dt):
    return np.dot(np.dot(np.dot(Phi, np.power(np.diag(mu), t/dt)), np.linalg.pinv(Phi)), X1[:,0])

print(np.real(x(27,Phi,mu,8)))
print(Precios[:,-1])

[ 27.47004377 122.75318954  46.6544748   15.01665036  43.24584361
  25.33307976 188.12369793  85.84464392  59.58769253 149.84002223
  13.97683183   2.57225662  15.79425227  13.52818845]
[ 26.31500053 114.33999634  55.47999954  33.79449844  42.95999908
  38.90050125 180.22999573  76.26999664  66.02999878 131.56787109
  16.00066757   2.86999989  27.07999992  19.07999992]


In [49]:
#Funcion que define que posicion voy a tomar

precios_futuros = np.real(x(27,Phi,mu,8))
precios_actuales = Precios[: , 18]

def posicion(precios_actuales, precios_futuros):

    precio_cartera_actual = np.sum(precios_actuales) / len(precios_actuales)
    precio_cartera_futuro = np.sum(precios_futuros) / len(precios_futuros)

    if precio_cartera_actual >= precio_cartera_futuro:
        posicion_actual = 'short'
    elif precio_cartera_actual < precio_cartera_futuro: 
        posicion_actual = 'long'

    return posicion_actual, precio_cartera_futuro

nueva_posicion = posicion(precios_actuales, precios_futuros)

print(nueva_posicion[0])

short


In [ ]:
#Funcion que vaya guardando la acertividad de el algoritmo

#Defino dos variables enteras, una denotara las veces que mi algoritmo hizo una buena prediccion, la otra contara las predicciones totales
casos_favorables = 0
casos_totales = 0

def acertividad(precios_predecidos, precios_actuales, precios_pasados):

    casos_totales += 1

    if precios_actuales >= precios_pasados:
        if precios_predecidos >= precios_pasados:
            casos_favorables += 1 
        #elif precios_predecidos < precios_pasados: casos_favorables += 0
    elif precios_actuales < precios_pasados:
        if precios_predecidos < precios_pasados:
            casos_favorables += 1
        #elif precios_predecidos >= precios_pasados: casos_favorables += 0